In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pennylane import numpy as np
from sklearn.decomposition import PCA
import single_model_train
import qcnn_estimator
import data

In [2]:
samples = data.data_load_and_split(["rock", "country"])

estimator = qcnn_estimator.QCNNEstimator()
pipeline = Pipeline(steps=[
  ("scaler", MinMaxScaler((0, np.pi / 2))),
  ("pca", PCA(n_components=8)),
  ("model", estimator)
])

In [3]:
#--- Grid Search for hyperparameters
grid_params = { 'model__stride_c':list(range(2,4)),
                'model__filter_p':["!*"]
              }

grid = GridSearchCV(pipeline, grid_params, cv=5, n_jobs=8, verbose=True, refit=True)

In [4]:
grid.fit(samples.x_train, samples.y_train.values)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


Loss at step 0: 0.6982115337436056
Loss at step 0: 0.6946484653113973
Loss at step 0: 0.7014424058368905
Loss at step 0: 0.7206898357651145
Loss at step 0: 0.7206898357651145
Loss at step 0: 0.7084598267319785
Loss at step 0: 0.7084598267319785
Loss at step 0: 0.7014424058368905
Loss at step 25: 0.5175874796969843
Loss at step 25: 0.5766912484411302
Loss at step 25: 0.547515855202973
Loss at step 25: 0.5450375472322369
Loss at step 25: 0.5474961806936178
Loss at step 25: 0.5766912484411302
Loss at step 25: 0.5450375472322369
Loss at step 25: 0.5175874796969843
Loss at step 50: 0.5054262674786305
Loss at step 50: 0.539391756883091
Loss at step 50: 0.539391756883091
Loss at step 50: 0.5398598784153708
Loss at step 50: 0.5434634352179061
Loss at step 50: 0.5434634352179061
Loss at step 50: 0.5349856339084472
Loss at step 50: 0.5054262674786305
Loss at step 75: 0.5024735164866788
Loss at step 75: 0.5372356362528039
Loss at step 75: 0.5372356362528039
Loss at step 75: 0.5377049376264014
Los

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler',
                                        MinMaxScaler(feature_range=(0,
                                                                    1.5707963267948966))),
                                       ('pca', PCA(n_components=8)),
                                       ('model', QCNNEstimator(stride_c=2))]),
             n_jobs=8,
             param_grid={'model__filter_p': ['!*'], 'model__stride_c': [2, 3]},
             verbose=True)

In [5]:
# Get the best model
best_circuit = grid.best_estimator_
# Print the best hyperparameters
print('=========================================[Best Hyperparameters info]=====================================')
# summarize best
print('Best score: %.3f'  % grid.best_score_)
print('Best Config: %s' % grid.best_params_)
print('==========================================================================================================')

=========================================[Best Hyperparameters info]=====================================
Best score: 0.736
Best Config: {'model__filter_p': '!*', 'model__stride_c': 2}


In [6]:
best_circuit.score(samples.x_test, samples.y_test.values)

0.75

In [10]:
import pandas as pd
results = pd.DataFrame(grid.cv_results_)

In [14]:
import os
os.makedirs('results/', exist_ok=True)

In [27]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
print(f"results/out_{date}.csv")

results/out_2024-03-25_10-21-13.csv


In [28]:
results.to_csv(f"results/out_{date}.csv", index=False)

In [30]:
pd.read_csv("results/out_2024-03-25_10-21-13.csv")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__filter_p,param_model__stride_c,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,29.524556,1.148249,0.062293,0.011935,!*,2,"{'model__filter_p': '!*', 'model__stride_c': 2}",0.928571,0.5,0.785714,0.857143,0.607143,0.735714,0.159079,1
1,22.381664,8.725236,0.058412,0.025182,!*,3,"{'model__filter_p': '!*', 'model__stride_c': 3}",0.928571,0.5,0.785714,0.857143,0.535714,0.721429,0.172615,2
